<a href="https://colab.research.google.com/github/RL44AI/Seminario/blob/main/Seminario_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Ruben Lima Garcia  <br>
Url: https://github.com/RL44AI/Seminario/blob/main/Seminario_Algoritmos.ipynb<br>
Problema:
>2. Organizar los horarios de partidos de La Liga<br>

Descripción del problema:(copiar enunciado)
Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios de forma que maximice
la audiencia.

(*) La respuesta es obligatoria





                                        

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Respuesta

Tomando los datos que tenemos que son la cantidad de partidos y los horarios dispobibles podemos obtener la cantidad de posibilidades sin tener en cuenta ninguna restricción.   

Datos:

      ------- Horarios ------
      Viernes 20hs
      Sabado  12,16,18,20
      Domingo 12,16,18,20
      Lunes   20hs
      ------- Partidos ------
      Tenemos 10 partidos en total, ya que tenemos 20 equipos.

De esto, podemos inferir que el partido 1 tiene 10 opciones de horario, el partido 2 tambien tiene 10, y asi sucesivamente ya que el problema permite aunque con un costo, los multiples partidos en horarios.
El problema tambien muestra una tabla con hasta 8 conincidencias de horarios, reduciendo un 80% de audiencia si pusieramos el problema como maximo 8 coincidencias de horario, el valor no cambiaría mucho asi que podemos decir que se acerca al de sin restricciones para los 10 partidos.

Para hacer el calculo usamos los partidos posibles y los horarios posibles y hacemos una ecuacion que es la (horarios posibles) ^ (cantidad de partidos) lo que nos deja con 10^10

In [1]:
A= 10**10
print(A)

10000000000


A partir de este acercamiento podemos obtener tambien las posibilidades que tienen los partidos utilizando todas las restricciones. En este caso, aunque, tenemos varias hits negativos al hacer X o Y no son restricciones como tal, por lo que la unica real, son los horarios. El hecho de que tenemos que tener tanto 1 partido viernes, como 1 partido lunes.

Para lograr obtener esto, podemos basarnos en que ya sabemos el total sin restricciones y restamos las posibilidades para las restricciones.
Sabemos que tenemos que cumplir que 1 partido que no este en viernes (A) y 1 partido que no este en lunes (B) y tambien esta la posibilidad de no tener ningun partido ni lunes o viernes suponiendo que todos vayan al mismo horario (C). Existe que todos vayan tanto a lunes como viernes pero no cambia las posibilidades ya que esta inlcuido dentro del calculo general de las posibilidades.

Formula = cantidad de horarios ^ cantidad de partidos
A= (9 horarios posibles) ^ (10 partidos disponibles ) = 9^10
B= (9 horarios posibles) ^ (10 partidos disponibles ) = 9^10
C= (8 horarios posibles) ^ (10 partidos disponibles ) = 8^10.

Entonces sumando todas estas restricciones nos da un valor de posibilidades donde no se cumple la restriccion pedida, este valor lo restamos a las posibilidades totales obtenidas anteriormente y ya obtenemos el valor buscado.



In [6]:
PartidosSinRestricciones = 10**10
PartidosSinViernes=9**10
PartidosSinLunes=9**10
PartidosSinLunesyViernes=8**10 #Usando la formula de la union, procedemos a evitar la repeticion de los casos y restamos los que no tienen ningun viernes ni lunes.
PartidosConRestricciones=(PartidosSinViernes+PartidosSinLunes)-PartidosSinLunesyViernes
PosibilidadesTotales = PartidosSinRestricciones - PartidosConRestricciones
print(PosibilidadesTotales)

4100173022


Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Debido a los tipos de datos utilizados como primera prueba intentare utilizar listas y matrices. Esto lo cambiaré si mas adelante veo que es muy lento o pruebo una solucion mas rapida con otro tipo de estructura de datos.

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

Para obtener la función objetivo debemos ver que es lo que tenemos para calcularla. La misión es maximizar la cantidad de audiencia, sabemos que tenemos una cantidad de equipos, de horarios disponibles y 1 restricción fija. Sabemos tambien que dependiendo de la cantidad de partidos en un mismo horario y/o de equipos que juegan entre si hay un valor que va variando respecto a una tabla de porcentajes. Por lo tanto podriamos empezar a describir las variables con las que contamos:

    Partidos: Sabemos que tenemos que tener 10 partidos en total, por lo que Partidos =[partido1,partido2,partido3,etc...]

    Horarios: [H1,H2,H3 etc... ] en donde estas Hs representan sabado a las 12, sabado a las 16 y el resto de horarios disponibles

    X = una matriz de valores para la asignacion de partidos, como cada partido se puede asignar solamente 1 vez a un horario, no puede estar en multiples.
    
    AudienciaBase = [P1 = 2.0,   # partido “A vs A”, P2= 1.3,   # “A vs B”, P3= 1.0,   # “A vs C”, P4: 0.9,   # “B vs B”, P5: 0.75   # “B vs C”]

    CoeficienteHorario  [0.4 # Viernes a las 20hs, 0.55 # sabado a las 12 hs, etc ]

    CoeficienteCoincidencia [0, 25% #1 partido, 45% #2 partidos, %60 #3 partidos] al saber que el problema dice que se reduce en esta cantidad entonces el valor real a utilizar sería 1-CoeficienteCoincidencia

(*)¿Es un problema de maximización o minimización?

Es totalmente un problema de maximización, ya que queremos maximizar la audiencia.

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

empezamos por saber lo que tenemos que lograr y los valores que ya definimos arriba, en como debería ir la ecuación.

Al tener 10^10 posibles soluciones totales, y luego como ya vimos, la reducidas por las restricciones, tenemos que tener un espacio de soluciones PosiblesSoluciones, que va a tener tanto valores tanto en X como en Y y que va a ser del tamaño total de las soluciones posibles.

Debemos contar cada partido cuando este, este en la posicion que queremos respecto a la x y Y es decir, va a ser 1 en una posicion y 0 en las demas debido a esto, necesitamos una sumatoria de 0 a 10 en X y luego complementarlo con una sumatoria de 0 a 10 en Y para ir recorriendo toda la matriz de soluciones.

Tenemos el valor de AudienciaBase, que es el valor que tienen los partidos AA, AB etc. ese valor sabemos que tambien viene modificado por el coeficiente de horario y asimismo por el coeficiente de coincidencia. Este coeficiente nos ayuda a determinar cuantos partidos hay en 1 horario por lo que tenemos que sumar todos, eso se convierte en una sumatoria de los partidos del mismo horario exceptuando, el que estamos revisando claro está.

Entonces la formula quedaria algo asi
    x=1∑10​y=1∑10​MatrizSolucionesxy​⋅AudienciaBase(px​)⋅CoeficienteHorario(fjy)⋅CoeficienteCoincidencia(k=1∑10​MatrizSolucionesky​−1)


En lugar de tirar todo el codigo de una vez, como normalmente me gusta hacerlo. debido a la complejidad de hacer varias iteraciones de 10^10. Intentare hacer un batch de prueba de 5^5 opciones. Que me puede ayudar a verificar si el codigo esta estructurado bien para un algoritmo de fuerza bruta y si nos provee de una solucion que nos parezca razonable.

Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

(*)Calcula la complejidad del algoritmo

Respuesta

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

Aplica el algoritmo al juego de datos generado

Respuesta

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta